In [1]:
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF

#from modAL.models import ActiveLearner

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
import math

%matplotlib inline

In [2]:
# cleaned version of the data after removing outliers and negative values
path='/workspace/malathi/steam_turbine_stage_1_POC/notebooks/version_1/df_sorted.csv'
data=pd.read_csv(path)
data.shape

(7077, 6)

In [3]:
INPUT_COLUMNS=[ 'expansion ratio','corrected speed','ImpulseStage_CurrentStroke']
OUTPUT_COLUMNS=['ImpulseStage_Power', 'Impulse_Discharge_Temperature', 'corrected mass flow']

In [9]:
# predictions for 'Impulsestage_Power'
X=data.drop(OUTPUT_COLUMNS,axis=1)
y=data['ImpulseStage_Power']
print(X.shape,y.shape)


(7077, 3) (7077,)


In [5]:
# XX=X.values.reshape(-1,3)
# yy=y.values.reshape(-1,1)
print(XX.shape,yy.shape)

(7077, 3) (7077, 1)


In [10]:
X_pool, X_test, y_pool, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
#X_pool, X_test, y_pool, y_test = X_pool.reset_index(drop=True), X_test.reset_index(drop=True), y_pool.reset_index(drop=True), y_test.reset_index(drop=True)
# random state 1 5 iterations
# random state 2 20 iteration

In [14]:
X_pool, X_test, y_pool, y_test = X_pool.reset_index(drop=True), X_test.reset_index(drop=True), \
y_pool.reset_index(drop=True), y_test.reset_index(drop=True)

In [19]:
# take 100 points from the pool, train them
train_idx=list(range(100))
# keep extending this range to get more ambiguous points
unknown_idx=list(range(100,1000))

In [20]:
X_train=X_pool.iloc[train_idx]
y_train=y_pool.iloc[train_idx]

In [37]:
X_unk = X_pool.iloc[unknown_idx]

In [27]:
estimator=GaussianProcessRegressor()
model=estimator.fit(X_train,y_train)

In [29]:
# simply for verification
# a= predictions # b =std
#a,b=model.predict(X_test,return_std=True)

/usr/local/lib/python3.8/dist-packages/sklearn/gaussian_process/_gpr.py:362: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [60]:
def find_most_ambiguous(model,data):
    _, std = model.predict(data, return_std=True)
    query_idx = np.argmax(std)
    print(query_idx)
    return query_idx, X_pool.iloc[query_idx]

In [61]:
# find ambiguous points
idx,ambiguous_data=find_most_ambiguous(model,X_unk)

56


/usr/local/lib/python3.8/dist-packages/sklearn/gaussian_process/_gpr.py:362: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [62]:
ambiguous_data

expansion ratio                 7.427656
corrected speed               143.980509
ImpulseStage_CurrentStroke     10.590000
Name: 56, dtype: float64

In [55]:
# for range(100,1000) in x_pool unambiguous point is 56

In [69]:
X_unk1=X_unk.reset_index(drop=True)

In [71]:
X_unk1.iloc[56]

expansion ratio                 1.644108
corrected speed               126.675307
ImpulseStage_CurrentStroke     36.350000
Name: 56, dtype: float64